In [7]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [5]:
!pip install tensorflow


In [17]:
# Open and read the text file
try:
    with open('intents.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()  # Read lines instead of JSON parsing

        if not lines:
            raise ValueError("Error: intents.txt file is empty!")

except FileNotFoundError:
    print("Error: intents.txt file not found!")
    exit()

# Initialize lists
training_sentences = []
training_labels = []
labels = []
responses = []

# Process each line in the text file
for line in lines:
    line = line.strip()
    if not line:
        continue  # Skip empty lines

    try:
        # Assume format: tag|pattern|response
        tag, pattern, response = line.split('|')

        training_sentences.append(pattern.strip())
        training_labels.append(tag.strip())

        if tag.strip() not in labels:
            labels.append(tag.strip())

        responses.append(response.strip())

    except ValueError:
        print(f"Skipping invalid line: {line}")  # Debugging for incorrect format

# Count classes
num_classes = len(labels)

# ✅ Debugging: Print some extracted data
print("Training sentences:", training_sentences[:5])
print("Labels:", labels)
print("Responses:", responses[:2])  # Preview first 2 responses


Training sentences: ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay']
Labels: ['greeting', 'goodbye', 'thanks', 'about', 'name', 'help', 'createaccount', 'complaint']
Responses: ['Hello', 'Hi']


In [19]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [21]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

C:\Users\ashis\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.1733 - loss: 2.0787 
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2143 - loss: 2.0754
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2143 - loss: 2.0734
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2039 - loss: 2.0726
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2143 - loss: 2.0706
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2143 - loss: 2.0690
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2039 - loss: 2.0690
Epoch 8/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2143 - loss: 2.0674
Epoch 9/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2143 - loss: 2.0663
Epoch 10/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2039 - loss: 2.0665
Epoch 11/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2143 - loss: 2.0641
Epoch 12/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2143 - l

In [25]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=chat_model.

In [27]:
# Save model in Keras format (Recommended)
model.save("chat_model.keras")

# OR save in HDF5 format (if needed)
model.save("chat_model.h5")


In [29]:
import pickle

# Save the trained model in the recommended format
model.save("chat_model.keras")  # Use .keras instead of a plain filename

# Save the fitted tokenizer
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the fitted label encoder
with open("label_encoder.pickle", "wb") as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

print("Model, tokenizer, and label encoder saved successfully!")


Model, tokenizer, and label encoder saved successfully!


In [ ]:
import random

# Open the intent.txt file and check if it's empty
try:
    with open('intents.txt', 'r', encoding='utf-8') as file:
        lines = file.read().strip().split('\n')  # Read and split the file by newlines

        if not lines:  # Check if the file is empty
            raise ValueError("Error: intent.txt file is empty!")

        # Parse the intent data from the lines
        intents = {}
        for line in lines:
            parts = line.split('|')  # Split by pipe character
            if len(parts) == 3:
                tag, pattern, response = parts
                if tag not in intents:
                    intents[tag] = {'patterns': [], 'responses': []}
                intents[tag]['patterns'].append(pattern)
                intents[tag]['responses'].append(response)

except FileNotFoundError:
    print("Error: intent.txt file not found!")
    exit()

except ValueError as ve:
    print(ve)
    exit()

# ✅ Debugging: Print the parsed intent data
print("Parsed Intents Data:", intents)

# Example usage: Chat simulation
def chat():
    while True:
        print("User: ", end="")
        inp = input()

        if inp.lower() == "quit":
            break

        # Find matching intent (simple pattern match)
        response = "Sorry, I didn't understand that."
        for tag, data in intents.items():
            if any(pattern.lower() in inp.lower() for pattern in data['patterns']):
                response = random.choice(data['responses'])
                break

        print("ChatBot:", response)

print("Start chatting with the bot (type 'quit' to stop)!")

# Start the chatbot
chat()


Parsed Intents Data: {'greeting': {'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'], 'responses': ['Hello', 'Hi', 'Hi there', 'Hello', 'Hi']}, 'goodbye': {'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']}, 'thanks': {'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]}, 'about': {'patterns': ['Who are you?', 'What are you?', 'Who you are?'], 'responses': ["I'm Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot", "I'm Joana, an Artificial Intelligent bot"]}, 'name': {'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Joana.', "I'm Joana!", 'Just call me Joana']}, 'help': {'patterns': ['Could you help me?', 'give me a hand please', 'Can you help?', 'What can you do for me?', 'I need a support', 'I need a help', 'sup

 hi


ChatBot: Hi
User: 

 hello


ChatBot: Hello
User: 

...

ChatBot: Sorry, I didn't understand that.
User: 

 you cant help me 


ChatBot: Sorry, I didn't understand that.
User: 

 have a complaint


ChatBot: Please provide us your complaint in order to assist you
User: 